In [19]:
import numpy as np
import os
import pandas as pd

In [101]:
# read data
df = pd.read_pickle(os.getcwd() + '\\data.pkl')
df.head()

,id,label,qID,bm25,idf,otf,jm,laplace,qtf,perc,docLen
0,85_AP891002-0245,1,85,0.742271,0.892695,0.602695,0.399030,1.197129,12,0.025586,469
1,85_AP890107-0129,1,85,1.180927,1.535395,1.240963,1.421737,0.440987,9,0.060000,150
2,85_AP890807-0114,1,85,-0.890010,-0.838847,-0.233771,-0.805927,-0.641625,4,0.021277,188
3,85_AP890819-0113,1,85,0.480031,0.761688,0.288280,0.633516,0.681506,13,0.036313,358
4,85_AP891017-0042,1,85,-1.932698,-1.907871,-1.229910,-1.830729,-0.934974,4,0.011331,353


In [102]:
# check number of rel and irr docs
df['label'].value_counts()

0    25000
1     1832
Name: label, dtype: int64

In [103]:
# get all query IDs
qID = df['qID'].unique()
qID

array([ 85,  59,  56,  71,  64,  62,  93,  99,  58,  77,  54,  87,  94,
       100,  89,  61,  95,  68,  57,  97,  98,  60,  80,  63,  91],
      dtype=int64)

In [123]:
np.random.shuffle(qID)
splits = np.split(qID, 5)
splits

[array([99, 62, 68, 63, 60], dtype=int64),
 array([ 89, 100,  85,  93,  94], dtype=int64),
 array([58, 56, 54, 59, 91], dtype=int64),
 array([98, 57, 87, 71, 80], dtype=int64),
 array([97, 77, 61, 64, 95], dtype=int64)]

In [207]:
from sklearn import linear_model, metrics, svm, tree

In [206]:
testAvg = 0
for i in range(5):  # 5-fold CV
    testQ = splits[i]  # select queries for testing
    trainQ = np.setdiff1d(qID, testQ)  # select queries for trainig
    
    train = df[df['qID'].isin(trainQ)].copy()
    test = df[df['qID'].isin(testQ)].copy()
    
#     print(train.shape, test.shape, train.shape[0] + test.shape[0])

    trainX = train.loc[:, ('bm25', 'idf', 'otf', 'jm', 'laplace', 'perc', 'docLen')]
    trainY = train.loc[:, 'label']
    
    testX = test.loc[:, ('bm25', 'idf', 'otf', 'jm', 'laplace', 'perc', 'docLen')]
    testY = test.loc[:, 'label']

#     print(trainX.shape, trainY.shape)
#     print(testX.shape, testY.shape)

    lm = linear_model.LinearRegression()
    lm.fit(trainX, trainY)  # fit model
    
    train.loc[:, 'pred'] = lm.predict(trainX)
    test.loc[:, 'pred'] = lm.predict(testX)

    # DT
#     dt = tree.DecisionTreeClassifier()
#     dt.fit(trainX, trainY)  # fit model
    
#     train.loc[:, 'pred'] = dt.predict(trainX)
#     test.loc[:, 'pred'] = dt.predict(testX)

#     train.loc[:, 'pred'] = dt.predict_proba(trainX)
#     test.loc[:, 'pred'] = dt.predict_proba(testX)

#     print(metrics.accuracy_score(trainY, trainPred))
#     acc = metrics.accuracy_score(testY, test['pred'])
#     print('Model', i + 1, 'acc:', acc)
#     testAvg += acc
    
    print('writing')
#     f = [train, test]
#     writeToFile(pd.concat(f), i)
    writeToFile(train, i, 'train')
    writeToFile(test, i, 'test')
    
    print('----' * 5)
    
# print('Avg acc:', testAvg / 5)

writing
--------------------
writing
--------------------
writing
--------------------
writing
--------------------
writing
--------------------


In [211]:
testAvg = 0
for i in range(5): 
    testQ = splits[i]  # select queries for testing
    trainQ = np.setdiff1d(qID, testQ)  # select queries for trainig
    
    train = df[df['qID'].isin(trainQ)].copy()
    test = df[df['qID'].isin(testQ)].copy()
    
#     print(train.shape, test.shape, train.shape[0] + test.shape[0])

    trainX = train.loc[:, ('bm25', 'idf', 'otf', 'jm', 'laplace', 'perc', 'docLen')]
    trainY = train.loc[:, 'label']
    
    testX = test.loc[:, ('bm25', 'idf', 'otf', 'jm', 'laplace', 'perc', 'docLen')]
    testY = test.loc[:, 'label']
    
    model = svm.SVR(kernel = 'rbf')
    model.fit(trainX, trainY)
    
    train.loc[:, 'pred'] = lm.predict(trainX)
    test.loc[:, 'pred'] = lm.predict(testX)
    
    print('writing')
#     f = [train, test]
#     writeToFile(pd.concat(f), i)
    writeToFile(train, i, 'train')
    writeToFile(test, i, 'test')
    
    print('----' * 5)

writing
--------------------
writing
--------------------
writing
--------------------
writing
--------------------
writing
--------------------


In [200]:
def writeToFile(res, count, typ = None):
    fileName = None
    
    if typ is None:
        fileName = 'preds_' + str(count) + '.txt'
    else:
        fileName = 'preds_' + typ + str(count) + '.txt'
    
    rd = {i: [] for i in res['qID'].unique()}
    
    res['docID'] = res['id'].apply(lambda x: x.split('_')[1])
    
    for i in res.index:
        rd[res.loc[i, 'qID']].append((res.loc[i, 'docID'], res.loc[i, 'pred']))
          
    opFile = open(os.getcwd() + '\\ResFiles\\' + fileName, 'w')
    
    for i in rd:
        rd[i] = sorted(rd[i], key = lambda x: x[1], reverse = True)[: 1000]
        
#         print('writing', i)
        rank = 1
        
        for dat in rd[i]:
            line = str(i) + ' Q0 ' + dat[0] + ' ' + str(rank) + ' ' + str(dat[1]) + ' Exp\n'
            rank += 1
            opFile.write(line)
    
    opFile.close()